In [2]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, FactorRange, Whisker
from bokeh.models.glyphs import VArea
from bokeh.models.tools import HoverTool
import bokeh.palettes
from bokeh.transform import factor_cmap
import math 
import pandas as pd

df = pd.read_csv('Results/export_ram.csv')
print(df)


       runtime image_name ressource_limit  number_of_targets  repeat  \
0      default   go-httpd         default                  1       1   
1      default   go-httpd         default                  1      10   
2      default   go-httpd         default                  1      11   
3      default   go-httpd         default                  1      12   
4      default   go-httpd         default                  1      13   
...        ...        ...             ...                ...     ...   
15465    runsc     tomcat             min                  8       5   
15466    runsc     tomcat             min                  8       6   
15467    runsc     tomcat             min                  8       7   
15468    runsc     tomcat             min                  8       8   
15469    runsc     tomcat             min                  8       9   

       ram_usage                                           filepath  
0            390  /opt/auswertung/Raw/apache_benchmark_repeat35/.

In [3]:
# Exclude gvisor with httpd
filterd = df[(df.runtime != 'runsc')]
#print(filterd)
df_gvisor = df[(df.runtime == 'runsc')]
#print(df_gvisor)
df_gvisor = df_gvisor[(df_gvisor.image_name != 'httpd')]
#print(df_gvisor)

frames = [filterd, df_gvisor]
filterd = pd.concat(frames)
print(filterd)
filterd = filterd[(filterd.image_name != 'node-express')]
print(filterd)

       runtime image_name ressource_limit  number_of_targets  repeat  \
0      default   go-httpd         default                  1       1   
1      default   go-httpd         default                  1      10   
2      default   go-httpd         default                  1      11   
3      default   go-httpd         default                  1      12   
4      default   go-httpd         default                  1      13   
...        ...        ...             ...                ...     ...   
15465    runsc     tomcat             min                  8       5   
15466    runsc     tomcat             min                  8       6   
15467    runsc     tomcat             min                  8       7   
15468    runsc     tomcat             min                  8       8   
15469    runsc     tomcat             min                  8       9   

       ram_usage                                           filepath  
0            390  /opt/auswertung/Raw/apache_benchmark_repeat35/.

In [4]:
# Remove -1 Values
filterd = filterd[(filterd.ram_usage != -1)]
print(filterd)

grouped = filterd

       runtime image_name ressource_limit  number_of_targets  repeat  \
0      default   go-httpd         default                  1       1   
1      default   go-httpd         default                  1      10   
2      default   go-httpd         default                  1      11   
3      default   go-httpd         default                  1      12   
4      default   go-httpd         default                  1      13   
...        ...        ...             ...                ...     ...   
15465    runsc     tomcat             min                  8       5   
15466    runsc     tomcat             min                  8       6   
15467    runsc     tomcat             min                  8       7   
15468    runsc     tomcat             min                  8       8   
15469    runsc     tomcat             min                  8       9   

       ram_usage                                           filepath  
0            390  /opt/auswertung/Raw/apache_benchmark_repeat35/.

In [5]:
# Fehlerrechnung

std = grouped.groupby(['runtime', 'image_name', 'ressource_limit', 'number_of_targets'])['ram_usage'].std().reset_index(name='std')
print(std)

mean = grouped.groupby(['runtime', 'image_name', 'ressource_limit', 'number_of_targets'])['ram_usage'].mean().reset_index(name='mean')
print(mean)

std['mean'] = mean[['mean']]
grouped_std_mean = std
print(grouped_std_mean)

# 95% Vertrauensintervall
grouped_std_mean['abweichung_in_prozent'] = grouped_std_mean['std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['mean']
grouped_std_mean['konf_intervall_unten'] = grouped_std_mean['mean'] - grouped_std_mean['std']  / math.sqrt(35) * 1.6909
grouped_std_mean['konf_intervall_oben'] = grouped_std_mean['mean'] + grouped_std_mean['std']  / math.sqrt(35) * 1.6909

grouped_std_mean = grouped_std_mean.round(2)
print(grouped_std_mean)

#summe_delta_s_quadrat = filterd['delta_s_quadrat'].sum()
#standartabweichung = math.sqrt(summe_delta_s_quadrat/199)
#print('standartabweichung: ' + str(standartabweichung))

# Formel t Verteilung: http://www.statistics4u.info/fundstat_germ/cc_confidence_interval.html
# T Verteilungs tabelle: https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf
# Nachschauen:
# n -1 = 39 <- richtig?
# a = 0,005 bei 99,5% Wahrscheinlichkeit
# Werte 38-> 2,712 40-> 2,704 ~ 39 -> 2,7105
# neu: 39 -> 2.7079
# Formel:
# Erwartungswert +- Standartabweichung / sqrt(40) * 2,7105

     runtime image_name ressource_limit  number_of_targets         std
0    default   go-httpd         default                  1    5.912101
1    default   go-httpd         default                  2    5.888602
2    default   go-httpd         default                  3    8.177958
3    default   go-httpd         default                  5   17.502077
4    default   go-httpd         default                  8   45.178823
..       ...        ...             ...                ...         ...
409    runsc     tomcat             min                  3    5.830664
410    runsc     tomcat             min                  5   22.975800
411    runsc     tomcat             min                  8   35.269193
412    runsc     tomcat             min                 13   59.144560
413    runsc     tomcat             min                 21  103.428918

[414 rows x 5 columns]
     runtime image_name ressource_limit  number_of_targets         mean
0    default   go-httpd         default             

In [8]:
######
# Leistung indivduell
####

source = ColumnDataSource(pd.DataFrame(grouped_std_mean))
#print(source.data)

image_names = list( dict.fromkeys(source.data['image_name'].tolist()))
#print(image_names)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

color_dict = {'default': 0, 'kata': 2, 'katafc': 3, 'runsc': 4, 'runnc': 5} 
legend_dict = {'default': 'Docker (runc)', 'kata': 'Kata', 'katafc': 'Kata FC', 'runsc': 'gVisor (runsc)', 'runnc': 'Nabla (runnc)'} 
dash_dict = {'default': 'solid', 'max': 'dashed', 'min': 'dotted'}
for image_name in image_names:
    p = figure(x_axis_type='linear', width=1500, height=1100)
    filterd=grouped_std_mean[grouped_std_mean['image_name'] == image_name]
    
    source = ColumnDataSource(pd.DataFrame(filterd))
    #print(source.data)

    runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
    #print(runtimes)

    ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
    #print(ressource_limits)
    
    colour_counter = 0
    for runtime in runtimes:
        filterd_rnt=filterd[filterd['runtime'] == runtime]
        
        source = ColumnDataSource(pd.DataFrame(filterd_rnt))
        #print(source.data)

        ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
        #print(ressource_limits)
        
        for limit in ressource_limits:
            filterd_limit=filterd_rnt[filterd_rnt['ressource_limit'] == limit]
        
            source = ColumnDataSource(pd.DataFrame(filterd_limit))
            #print(source.data)
            #glyph = VArea(x='number_of_targets', y1='min_requests_per_second', y2='max_requests_per_second', fill_color=bokeh.palettes.d3['Category20'][12][1], fill_alpha=0.3)
            #p.add_glyph(default_source, glyph)
            p.line(x='number_of_targets', y='mean', line_width=2, line_dash=dash_dict[limit], source=source, color=bokeh.palettes.d3['Category10'][6][color_dict[runtime]], legend_label=legend_dict[runtime] + ' - ' + limit)
            
            
            p.add_layout(
              Whisker(source=source, base='number_of_targets', upper="konf_intervall_oben", lower="konf_intervall_unten", level="overlay")
            )

        colour_counter = colour_counter + 1
    
    p.title.text ='Arbeitsspeicherverbrauch nach Anzahl der parallelen Container für ' + image_name
    p.xaxis.axis_label = 'Anzahl an parallelen Containern'
    p.yaxis.axis_label = 'Arbeitsspeicherverbrauch [MB]'
    p.x_range.start = 1
    p.xaxis.ticker = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
    p.legend.click_policy="hide"
    p.legend.location = "top_left"
    p.legend.background_fill_alpha = 0.5
    p.title.text_font_size = '16pt'
    p.legend.label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "13pt"

    p.yaxis.axis_label_text_font_size = "15pt"
    p.yaxis.major_label_text_font_size = "13pt"

    output_file("requests_pro_sec_anzahl_container_" + image_name + ".html")
    output_notebook()
    show(p)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [14]:
######
# Leistung indivduell
####

source = ColumnDataSource(pd.DataFrame(grouped_std_mean))
#print(source.data)

image_names = list( dict.fromkeys(source.data['image_name'].tolist()))
#print(image_names)

number_of_targets = list( dict.fromkeys(source.data['number_of_targets'].tolist()))
#print(image_names)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

#x = [ (image_name, ressource_limit, runtime) for image_name in image_names for ressource_limit in ressource_limits for runtime in runtimes ]
#print(x)

for image_name in image_names:
    for targets in number_of_targets:
        
        
        filterd=grouped_std_mean[grouped_std_mean['image_name'] == image_name]
        filterd=filterd[filterd['number_of_targets'] == targets]

        source = ColumnDataSource(pd.DataFrame(filterd))
        #print(source.data)

        runtimes = list(dict.fromkeys(source.data['runtime'].tolist()))
        #print(runtimes)

        ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
        #print(ressource_limits)

        x = [ (ressource_limit, runtime) for ressource_limit in ressource_limits for runtime in runtimes ]
        print(x)

        source = ColumnDataSource(data=dict(x=x, mean=source.data['mean'].tolist(), oben=source.data['konf_intervall_oben'].tolist(), unten=source.data['konf_intervall_unten'].tolist()))
        print(source.data)
        p = figure(x_range=FactorRange(*x), width=1000)

        # use the palette to colormap based on the the x[1:2] values
        #fill_color=factor_cmap('x', palette=bokeh.palettes.d3['Category10'][5], factors=runtime, start=1, end=2)

        #p.vbar(x='x', top='mean', source=source, width=0.70, color=fill_color)
        p.vbar(x='x', top='mean', source=source, width=0.70)

        p.add_layout(
          Whisker(source=source, base='x', upper="oben", lower="unten", level="overlay")
        )


        p.title.text ='Ram Verbrauch für Image ' + image_name + ' mit der Skalierung' + str(targets)
        p.xaxis.axis_label = 'Ressourcelimit & Runtime'
        p.yaxis.axis_label = 'Ram Verbrauch'

        p.xaxis.axis_label_text_font_size = '11pt'
        p.yaxis.axis_label_text_font_size = '11pt'
       


        output_notebook()

        show(p)

[('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')], 'mean': [418.4, 295.6, 285.2, 447.77, 748.31, 504.49, 297.46, 299.46, 295.11, 321.17, 328.6, 322.11, 1563.46, 321.51, 303.8], 'oben': [420.09, 296.42, 287.56, 449.77, 750.71, 508.06, 299.75, 300.53, 297.18, 323.2, 332.1, 324.1, 1569.71, 323.35, 305.82], 'unten': [416.71, 294.78, 282.84, 445.78, 745.92, 500.91, 295.16, 298.39, 293.05, 319.14, 325.1, 320.13, 1557.2, 319.68, 301.78]}


Loading BokehJS ...

[('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')], 'mean': [472.83, 356.63, 342.71, 665.06, 1267.63, 777.34, 358.77, 360.91, 353.37, 426.17, 447.0, 432.74, 1668.43, 486.66, 372.8], 'oben': [474.51, 358.66, 344.92, 667.43, 1270.22, 781.5, 360.57, 364.26, 355.81, 429.1, 449.32, 437.13, 1671.84, 489.95, 374.62], 'unten': [471.15, 354.59, 340.51, 662.69, 1265.04, 773.19, 356.98, 357.57, 350.94, 423.24, 444.68, 428.35, 1665.02, 483.36, 370.98]}


Loading BokehJS ...

[('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'katafc'), ('max', 'runnc'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'katafc'), ('min', 'runnc'), ('min', 'runsc')], 'mean': [531.06, 420.4, 415.34, 878.31, 1794.54, 1046.4, 430.57, 429.37, 426.23, 532.69, 574.09, 547.66, 1843.29, 684.26, 454.03], 'oben': [533.39, 422.88, 417.97, 881.94, 1797.39, 1051.36, 432.91, 432.54, 429.73, 535.68, 581.06, 550.14, 1848.68, 689.0, 455.95], 'unten': [528.72, 417.92, 412.72, 874.69, 1791.7, 1041.44, 428.23, 426.2, 422.73, 529.69, 567.11, 545.18, 1837.89, 679.52, 452.11]}


...

Loading BokehJS ...

[('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')], 'mean': [4435.94, 4681.4, 3406.8, 6245.29, 10681.71, 6619.17, 2348.83, 1760.83, 1581.74], 'oben': [4475.12, 4710.12, 3435.14, 6283.77, 10710.47, 6653.13, 2365.0, 1780.18, 1598.65], 'unten': [4396.77, 4652.68, 3378.46, 6206.8, 10652.96, 6585.22, 2332.65, 1741.48, 1564.84]}


Loading BokehJS ...

[('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')]
{'x': [('default', 'default'), ('default', 'kata'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'runsc'), ('min', 'default'), ('min', 'kata'), ('min', 'runsc')], 'mean': [7093.49, 7325.8, 5556.03, 9913.63, 15612.86, 10833.29, 3327.03, 2940.17, 2564.6], 'oben': [7164.36, 7392.9, 5621.88, 9977.35, 15623.82, 10901.12, 3355.38, 3037.09, 2594.16], 'unten': [7022.61, 7258.7, 5490.18, 9849.91, 15601.89, 10765.45, 3298.68, 2843.25, 2535.04]}


Loading BokehJS ...

In [15]:
print(grouped_std_mean.to_string())


     runtime      image_name ressource_limit  number_of_targets     std      mean  abweichung_in_prozent  konf_intervall_unten  konf_intervall_oben
0    default        go-httpd         default                  1    5.91    418.40                   0.40                416.71               420.09
1    default        go-httpd         default                  2    5.89    472.83                   0.36                471.15               474.51
2    default        go-httpd         default                  3    8.18    531.06                   0.44                528.72               533.39
3    default        go-httpd         default                  5   17.50    670.03                   0.75                665.03               675.03
4    default        go-httpd         default                  8   45.18    988.86                   1.31                975.94              1001.77
5    default        go-httpd         default                 13  104.37   1702.91                   1.75        

In [16]:
grouped_std_mean['docker_abweichung'] = 0

#print(grouped_std_mean.loc[ 273 , : ])

for index, row in grouped_std_mean.iterrows():
    docker_mean = grouped_std_mean.loc[(grouped_std_mean['runtime'] == 'default') & (grouped_std_mean['number_of_targets'] == row['number_of_targets']) & (grouped_std_mean['image_name'] == row['image_name']) & (grouped_std_mean['ressource_limit'] == row['ressource_limit']), ['mean']].reset_index().get_value(0, 'mean')
    #print(index)
    #print(row['mean'] * 100 / docker_mean)
    grouped_std_mean.loc[index, 'docker_abweichung'] = row['mean'] * 100 / docker_mean

grouped_std_mean = grouped_std_mean.round(2)

print(grouped_std_mean.to_string())

export_csv = grouped_std_mean.to_csv (r'Results/Ram_Gewichtung_Docker.csv', index = None, header=True, decimal=',')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


     runtime      image_name ressource_limit  number_of_targets     std      mean  abweichung_in_prozent  konf_intervall_unten  konf_intervall_oben  docker_abweichung
0    default        go-httpd         default                  1    5.91    418.40                   0.40                416.71               420.09             100.00
1    default        go-httpd         default                  2    5.89    472.83                   0.36                471.15               474.51             100.00
2    default        go-httpd         default                  3    8.18    531.06                   0.44                528.72               533.39             100.00
3    default        go-httpd         default                  5   17.50    670.03                   0.75                665.03               675.03             100.00
4    default        go-httpd         default                  8   45.18    988.86                   1.31                975.94              1001.77             100.0